In [9]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

num_records = 100
forced_delivered_count = int(num_records * 0.15)  # Force 15% of rows to be Delivered

# Generate random GPS coordinates within the Philippines
def random_coordinates():
    lat = round(np.random.uniform(5.0, 19.0), 6)
    lon = round(np.random.uniform(117.0, 126.0), 6)
    return f"{lat},{lon}"

raw_data = []

for i in range(1, num_records + 1):
    is_forced_delivered = i <= forced_delivered_count

    # Assign current and delivery coordinates
    if is_forced_delivered:
        # Make them close enough to simulate a real delivery
        base_lat = round(np.random.uniform(5.0, 19.0), 6)
        base_lon = round(np.random.uniform(117.0, 126.0), 6)
        current_coordinates = f"{base_lat},{base_lon}"
        delivery_coordinates = f"{round(base_lat + np.random.uniform(-0.002, 0.002), 6)}," \
                               f"{round(base_lon + np.random.uniform(-0.002, 0.002), 6)}"
    else:
        current_coordinates = random_coordinates()
        delivery_coordinates = random_coordinates()

    # Perishable logic
    perishable = np.random.choice(["Yes", "No"])
    if perishable == "Yes":
        temperature = round(np.random.uniform(2.0, 12.0), 2)
        temperature_issue = "Exceeded" if temperature > 7.0 else "Normal"
    else:
        temperature = round(np.random.uniform(10.0, 30.0), 2)
        temperature_issue = "N/A"

    # Dates
    order_date = datetime.now() - timedelta(days=np.random.randint(3, 6))
    delivery_date = order_date + timedelta(days=np.random.randint(1, 3))
    timestamp = datetime.now() - timedelta(minutes=np.random.randint(0, 1440))
    delay_by_days = (timestamp.date() - delivery_date.date()).days

    # Distance calculation
    curr_lat, curr_lon = map(float, current_coordinates.split(","))
    dest_lat, dest_lon = map(float, delivery_coordinates.split(","))
    distance = abs(curr_lat - dest_lat) + abs(curr_lon - dest_lon)

    # Final status logic
    if is_forced_delivered:
        status = "Delivered"
    elif delay_by_days > 1 and distance >= 0.01:
        status = "Delayed"
    elif distance < 0.01:
        status = np.random.choice(["Delivered", "Awaiting Pickup"])
    else:
        status = np.random.choice(["In Transit", "Awaiting Pickup"])

    raw_data.append({
        "timestamp": timestamp,
        "order_date": order_date,
        "delivery_date": delivery_date,
        "current_coordinates": current_coordinates,
        "delivery_coordinates": delivery_coordinates,
        "perishable": perishable,
        "temperature_celsius": temperature,
        "temperature_issue": temperature_issue,
        "status": status
    })

# Sort by timestamp
sorted_data = sorted(raw_data, key=lambda x: x["timestamp"])

# Assign RFID and format dates
for idx, record in enumerate(sorted_data, start=1):
    record["rfid"] = f"RFID{str(idx).zfill(3)}"
    record["timestamp"] = record["timestamp"].strftime("%Y-%m-%d %H:%M:%S")
    record["order_date"] = record["order_date"].strftime("%Y-%m-%d")
    record["delivery_date"] = record["delivery_date"].strftime("%Y-%m-%d")

# Final column order
columns_order = [
    "timestamp",
    "rfid",
    "order_date",
    "delivery_date",
    "current_coordinates",
    "delivery_coordinates",
    "perishable",
    "temperature_celsius",
    "temperature_issue",
    "status"
]

# Save files
df = pd.DataFrame(sorted_data)[columns_order]
df.to_csv("iot_data.csv", index=False)
df.to_json("iot_data.json", orient="records")

# Preview
df.head()

,timestamp,rfid,order_date,delivery_date,current_coordinates,delivery_coordinates,perishable,temperature_celsius,temperature_issue,status
0,2025-05-16 16:39:31,RFID001,2025-05-14,2025-05-15,"11.022535,123.149032","18.936667,124.886806",Yes,3.84,Normal,Awaiting Pickup
1,2025-05-16 17:12:31,RFID002,2025-05-13,2025-05-15,"18.511009,124.893501","15.818907,121.826855",Yes,10.24,Exceeded,In Transit
2,2025-05-16 17:36:31,RFID003,2025-05-13,2025-05-15,"18.760037,119.527383","13.025807,119.439012",Yes,9.00,Exceeded,In Transit
3,2025-05-16 17:40:31,RFID004,2025-05-12,2025-05-13,"8.536344,123.058721","8.534478,123.057676",No,22.86,N/A,Delivered
4,2025-05-16 18:11:31,RFID005,2025-05-12,2025-05-13,"10.157701,120.085836","12.800504,124.99346",Yes,9.56,Exceeded,Delayed
